# 求解代码

In [4]:
# 环境配置 & 全局参数
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

# ===== 物理常量 =====
g = 9.8  # 重力加速度 (m/s²)
rho_sea = 1025  # 海水密度 (kg/m³)
depth = 18  # 水深 (m)

# ===== 系统参数 =====
# 浮标 (圆柱体)
d_buoy, h_buoy = 2.0, 2.0  # 直径, 高度 (m)
m_buoy = 1000  # 质量 (kg)

# 钢管 (4节)
n_pipes = 4
l_pipe, d_pipe = 1.0, 0.05  # 单节长, 直径 (m)
m_pipe = 10  # 单节质量 (kg)

# 钢桶
l_bucket, d_bucket = 1.0, 0.3  # 长度, 外径 (m)
m_bucket = 100  # 总质量 (kg)

# 重物球
m_ball = 1200  # 质量 (kg)

# 锚链 (II型)
chain_type = "II"
l_chain = 22.05  # 总长 (m)
m_chain_per_m = 7  # 线密度 (kg/m)
link_length = 0.105  # 单环长 (m)
n_links = int(l_chain / link_length)  # 环数 ≈210

# 锚
m_anchor = 600  # 质量 (kg)

In [ ]:
def buoy_force(h_sub, v_wind):
    """
    计算浮标受力
    h_sub: 吃水深度 (m)
    v_wind: 风速 (m/s)
    """
    # 投影面积 (迎风面)
    S_wind = d_buoy * min(h_sub, h_buoy)  # 不超过总高度
    
    # 风力 (N)
    F_wind = 0.625 * S_wind * v_wind**2
    
    # 水下体积 (m³)
    V_sub = np.pi * (d_buoy/2)**2 * h_sub
    
    # 浮力 (N)
    F_buoy = rho_sea * g * V_sub
    
    # 净垂直力 (浮力 - 重力)
    F_net_y = F_buoy - m_buoy * g
    
    # 浮标倾斜角
    theta = np.arctan2(F_wind, F_net_y)
    
    return F_wind, F_net_y, theta

In [ ]:
def pipe_segment(Tx_in, Ty_in, length, diameter, mass):
    """
    单节钢管受力计算
    Tx_in, Ty_in: 上端输入张力分量 (N)
    """
    # 水下体积 (m³)
    V_pipe = np.pi * (diameter/2)**2 * length
    
    # 浮力 (N)
    Fb_pipe = rho_sea * g * V_pipe
    
    # 净重力 (水下)
    Fg_net = mass * g - Fb_pipe
    
    # 下端垂直张力 (N)
    Ty_out = Ty_in - Fg_net
    
    # 水平张力不变 (无水平外力)
    Tx_out = Tx_in
    
    # 本节钢管倾角
    theta = np.arctan2(Tx_out, Ty_out)
    
    return Tx_out, Ty_out, theta

In [ ]:
def bucket_system(Tx_in, Ty_in):
    """
    钢桶系统受力计算
    """
    # 总质量 (钢桶+设备+重物球)
    total_mass = m_bucket + m_ball
    
    # 水下体积 (m³)
    V_bucket = np.pi * (d_bucket/2)**2 * l_bucket
    
    # 浮力 (N)
    Fb_bucket = rho_sea * g * V_bucket
    
    # 净重力
    Fg_net = total_mass * g - Fb_bucket
    
    # 输出张力 (至锚链)
    Tx_out = Tx_in  # 水平分量不变
    Ty_out = Ty_in - Fg_net  # 垂直分量
    
    # 钢桶倾角
    theta_bucket = np.arctan2(Tx_out, Ty_out)
    
    return Tx_out, Ty_out, theta_bucket

In [ ]:
def chain_simulation(Tx_start, Ty_start, n_links, link_length):
    """
    锚链离散模拟
    """
    # 单链环参数
    m_link = m_chain_per_m * link_length  # 质量 (kg)
    V_link = np.pi * (0.01)**2 * link_length  # 体积 (等效半径1cm)
    Fb_link = rho_sea * g * V_link  # 浮力
    
    # 初始化
    positions = np.zeros((n_links+1, 2))  # 节点坐标
    positions[0] = [0, -18 + l_bucket]  # 起始点（钢桶下方）
    tensions = np.zeros((n_links, 2))  # 张力向量
    angles = np.zeros(n_links)  # 每个链环角度
    
    Tx, Ty = Tx_start, Ty_start
    
    # 逐个链环计算
    for i in range(n_links):
        # 链环净重力
        Fg_net = m_link * g - Fb_link
        
        # 下端张力
        Ty_next = Ty - Fg_net  # 垂直分量
        Tx_next = Tx  # 水平分量不变
        
        # 计算倾角
        theta = np.arctan2(Tx_next, Ty_next)
        
        # 更新位置
        dx = link_length * np.sin(theta)
        dy = link_length * np.cos(theta)
        positions[i+1] = positions[i] + [dx, dy]
        
        # 存储张力和角度
        tensions[i] = [Tx_next, Ty_next]
        angles[i] = theta
        
        # 传递到下个环
        Tx, Ty = Tx_next, Ty_next
    
    # 末端角度 (锚点处)
    if n_links >= 2:
        last_vec = positions[-1] - positions[-2]
        theta_anchor = np.arctan2(last_vec[1], last_vec[0])
    else:
        theta_anchor = angles[-1]
    
    return positions, angles, theta_anchor

In [ ]:
def solve_system(v_wind, h_guess=1.0):
    """完整系统求解"""
    def force_balance(h_sub):
        # 1. 浮标系统
        F_wind, F_net_y, theta_buoy = buoy_force(h_sub, v_wind)
        
        # 浮标传递给钢管的张力
        Tx_pipe = F_wind / np.sin(theta_buoy)  # 水平分量
        Ty_pipe = F_net_y / np.cos(theta_buoy)  # 垂直分量
        
        # 2. 钢管系统 (4节)
        pipe_angles = []
        Tx, Ty = Tx_pipe, Ty_pipe
        for i in range(4):
            Tx, Ty, theta = pipe_segment(Tx, Ty, l_pipe, d_pipe, m_pipe)
            pipe_angles.append(theta)
        
        # 3. 钢桶系统
        Tx, Ty, theta_bucket = bucket_system(Tx, Ty)
        
        # 4. 锚链系统
        chain_pos, chain_angles, theta_anchor = chain_simulation(
            Tx, Ty, n_links, link_length
        )
        
        # 计算残差 (锚点约束)
        anchor_residual = theta_anchor - np.deg2rad(16)
        
        return anchor_residual
    
    # 优化吃水深度以满足锚点角度约束
    sol = root_scalar(
        force_balance, 
        x0=h_guess, 
        bracket=[0.5, 2.5],  # 物理合理范围
        method='brentq'
    )
    
    h_optimal = sol.root
    return h_optimal

In [ ]:
def test_no_wind():
    """验证无风状态浮力平衡"""
    # 理论吃水深度
    h_theory = (m_buoy * g) / (rho_sea * g * np.pi * (d_buoy/2)**2)
    
    # 计算值
    h_calc = solve_system(v_wind=0)
    
    # 误差分析
    error = abs(h_calc - h_theory)
    assert error < 0.01, f"无风验证失败，误差{error:.4f}m"
    print("无风验证通过!")

In [ ]:
def force_closure_test(v_wind):
    """验证系统总力平衡"""
    h_sub = solve_system(v_wind)
    
    # 计算浮标风力
    S_buoy = d_buoy * min(h_sub, h_buoy)
    F_wind_total = 0.625 * S_buoy * v_wind**2
    
    # 计算系统总重力
    total_mass = (
        m_buoy + 4*m_pipe + 
        m_bucket + m_ball + 
        m_chain_per_m * l_chain + 
        m_anchor
    )
    
    # 计算系统总浮力
    V_total = (
        np.pi*(d_buoy/2)**2*h_sub + 
        4*np.pi*(d_pipe/2)**2*l_pipe +
        np.pi*(d_bucket/2)**2*l_bucket
    )
    Fb_total = rho_sea * g * V_total
    
    # 水平力平衡
    F_horizontal = F_wind_total  # 应等于锚点水平反力
    
    # 垂直力平衡
    F_vertical = Fb_total - total_mass * g  # 应≈0
    
    print(f"水平力平衡: {F_horizontal:.2f}N")
    print(f"垂直力不平衡量: {F_vertical:.2f}N (应接近0)")